## Lamberts Problem 

Algorithm for solving Lamberts problem as presented in "A Simple Lambert Algorithm" by Giulio Avanzini, Journal of Guidance, Navigation and Control. The problem is defined as trying to determine an orbit which has a specific transfer time connection two position vectors. 

### Step 1: Assign the problem geometry and transfer time

**Problem Geometry:**

1. $\rho$ - The ratio of radii, $r_1/r_2$. $r_1$ is defined as the initial position of the orbiting body and $r_2$ is the final position of the orbiting body.  
   - For interplanetary transfers, the position vectors can be determined from initial positions of the transfer vehicle and planetary ephemeris data for the final position (at the time of capture)
2. $\Delta\theta$ - The transfer angle, or the angle between $r_1$ and $r_2$
3. $\tau_{12}$ - The non-dimensional transfer time. ($\tau_{12} = t_{12}\sqrt{\mu/r_1^3}$)

### Step 2: Determine $\Delta\theta > \pi$

The geometry of permissible orbits changes based on the flight angle between the starting and ending positions. To determine which approach is appropriate, the flight angle needs to be gated before branching down either $\Delta\theta > \pi$ or $\Delta\theta \leq \pi$

### Step 3: Derive Limits for $e_T$: 

Ultimately, the transverse eccentricity of the transfer orbit will be varied. However, based on the flight angle, there are limitations on how $e_T$ can be varied. 

**Case 1 and 2: $\Delta\theta \leq \pi$**

For the case where the flight angle is less than $\pi$, $e_T < e_P$ or less than the value of $e_T$ that would make the orbit parabolic. The following equations define $e_P$ from the input variables: 

\begin{equation*}
c = r_1 + r_2 
\tag{1}
\end{equation*}

\begin{equation*}
e_F = \frac{r_1 - r_2}{c} 
\tag{2}
\end{equation*}

\begin{equation*}
e_P = (1 - e_F^2)^{1/2}
\tag{3}
\end{equation*}

As $e_T$ approaches $-\infty$, the velocity of the parabolic orbit connecting P1 and P2 approaches infinity as well. 

**Case 3 and 4: $\Delta\theta \leq \pi$**

While there is no limitation on the minimum transverse eccentricity in cases 1 and 2, cases 2 and 3 has a lower limit. Since in this region hyperbolic orbits are possible, then the geometry of a hyperbolic orbit limits the lower end of permissible $e_T$ values. For deeper discussion of this limitation, see Avanzini. The bounds for case 3 and 4 become $-e_H < e_T < e_P$ or: 

\begin{equation*}
-\frac{r_1 - r_2}{c}\sqrt{\frac{1-cos\Delta\theta}{1+cos\Delta\theta}} < e_T < \frac{1}{c}\sqrt{2r_1r_2(1-cos\Delta\theta)}
\tag{4}
\end{equation*}

### Step 4 and 5: Initial Guess and Inverse Transform

The Newton-Raphson iteration method is used by Avanzini to converge on a value of e_T. Before applying this method, the eccentricity is transformed into parameter x such that $x = \gamma(e_T)$. This will help avoid solutions where the transfer time is no longer defined. 

For the case where $\Delta\theta \leq \pi$ the transformation and inverse transformation are given in equations 4 and 5. 

\begin{equation*}
x = \frac{e_Pe_H}{e_P+e_H}log(\frac{e_P}{e_H}\frac{e_T + e_H}{e_P - e_T})
\tag{4}
\end{equation*}

\begin{equation*}
e_T = e_Pe_H\frac{X-1}{e_P+e_HX}
\tag{5}
\end{equation*}

where X is: 

\begin{equation*}
X = exp[(\frac{1}{e_H} + \frac{1}{e_P})x]
\tag{6}
\end{equation*}

For the case where $\Delta\theta \leq \pi$, equations 7 and 8 show the transformation and its inverse: 

\begin{equation*}
x = -e_Plog(1-\frac{e_T}{e_P})
\tag{7}
\end{equation*}

\begin{equation*}
e_T=e_P[1-exp(-\frac{x}{-e_P})]
\tag{8}
\end{equation*}

The Newton Raphson method begins by making an initial guess for the variable being solved for. In this case, that is x, which typically will be x = 0, Additionally, the number of iterations k will be set to 0 as well, which will track whether the iterator is converging in a reasonable number of iterations ($k_max)$ which will be determined by trial and error when implementing this solution. 

Once a value of x is guessed, it is passed into the inverse transform for the respective initial conditions. This value of $e_T$ can then be passed into the Kepler time equation. 

### Step 6: Kepler Time Equation

The value of the transverse eccentricity calculated using the guessed value for x can now be used to determine the time of flight it produces. This is done by passing it into the Kepler time of flight equation. This is the most involved step. 

Calculating the time of flight between P1 and P2 for requires both Mean Anomalies $M_1$ and $M_2$, which can be determined from the transverse eccentricity, and the mean motion n. The relationship is shown in equation 9. 

\begin{equation*}
t_{12} = \frac{M_2 - M_1}{n}
\tag{9}
\end{equation*}

First, the mean motion n is broken down in equation 10. 

\begin{equation*}
n = \sqrt{\frac{\mu}{a^3}}
\tag{10}
\end{equation*}

A is the semi-major axis of the transfer orbit, which is given by equation 11. 

\begin{equation*}
a(e_T) = frac{p_F - e_T\frac{r_1r_2}{c}sin(\Delta\theta)}{1-e^2_F - e^2_F}
\tag{11}
\end{equation*}

Where $p_F = a_F(1-e^2_F)$, $a_F = (r_1+r_2)/2$ and $e_F = |r_2 - r_2|/c$.

These equations can be used to determine the mean motion n. Next the mean anomalies of both points needs to be determined. First the angle between the chord unit vector **$i_c$** and the vector **$r_1$** must be computed, noted as $\omega_c$. Together with $e_T$, the argument of periapsis is calculated using equation 12. 

\begin{equation*}
\omega(e_T) = tan^{-1}(e_Fsin\omega_c + e_Tcos\omega_c, e_Fsin\omega_c - e_Tcos\omega_c)
\tag{13}
\end{equation*}

Where $tan^{-1}$ is the four-quadrant inverse tangent function. With the argument of periapsis calculated, the mean anomaly for either point is given by $v_1 = - \omega$ and $v_2 =\Delta\theta - \omega$

The last value that is required to determine $M_1$ and $M_2$ is the eccentricity vector given by equation 13.

\begin{equation*}
e(e_T) = (e_Fcos\omega_c - e_Tsin\omega_c, e_Fsin\omega_c + e_Tcos\omega_c, 0)
\tag{14}
\end{equation*}

The kepler time equation can than be used to determine the mean anomaly $M = atan2(\frac{\sqrt{1-e^2}sin v}{1+e cos v}, \frac{e + cos v}{1+e cos v}) - \frac{\sqrt{1-e^2}sin v}{1+e cos v}$. 

After calculating $t_{12}$ from equation 9, it needs to be dimensionalised by multiplying it by $n_{ref} = \sqrt{\mu/r^3_1}$, $\tau_{12} = t_{12}n_{ref}$.

### Step 7: Determine convergence  

The next step is to determine whether the time of flight converges to the expected time of flight. First, we need to take the log of the calculated time of flight $y = log\tau_{12}$, and subtract this value from the expected value: 

\begin{equation*}
|y - log\tau^S_{12}| < \epsilon
\tag{15}
\end{equation*}

If the difference is less than the desired accuracy, the process can be exited. Otherwise, proceed to the next step. 

### Step 8: Check max convergence level

If the calculated $\tau_{12}$ does not converge within the desired tolerance, then the next step is to check if the number of iterations exceeds the max amount. If so, then the algorithm is not converging and should be exited. 

### Step 9: Increment the parameter

If the number of iterations does not exceed the max (after step 8), then the initial guess should be incremented by some ∆x. 

\begin{equation*}
x' = x_o + \Delta x
\tag{16}
\end{equation*}

### Step 10: Take inverse transform again and re-calculate the Kepler Time Equation

Using equation 8, again find the transverse eccentricity with the incremented value of x ($e_T$').

Using this value, find a new $\tau_{12}'$ and $y'$ value using the same approach as in step 7 with the new value for $e_T$'.

### Step 10: Newton-Raphson iteration

Using the Newton-Raphson method, the guess for x_{k+1} will be calculated. 

\begin{equation*}
dy/dx = (y'-y)/\Delta x
\tag{17}
\end{equation*}

\begin{equation*}
x_{k+1} = x_k + (y^S - y)/(dy/dx)
\tag{18}
\end{equation*}

The new value for $x_k$ will be used as the new guess for the next iteration, returning to step 5.